<img 
    style="position: absolute; 
           left: 60%; 
           top: 0; /* Added to ensure proper positioning */
           height: 900px; 
           width: 40%; /* Maintain the original width */
           object-fit: cover; /* Adjust if necessary */
           clip-path: inset(0px 50px 0px 50px round 10px);" 
    src="_data\imagedb\image_14570_s_a.png" 
/>
</figure>


<h1 style="width: 60%; color: #EC6842; font-size: 55pt;">
    <Strong>
        Streetscapes
        <h2 style="color: #EC6842; font-size: 40pt;"> 
            <Strong>
                Urban classification using street-level images and its embeddings
            </Strong>
        </h2>
    </Strong>
</h1>

<h3 id="Background"><B>
    Rationale for the project<a class="anchor-link" href="#Background">&#182;</a>
    </B>
</h3>
<p style="text-align: justify; width: 60%; font-weight: normal;">
     Structures that make optimal use of the material they are made of reduces the cost and environmental impact of their construction as the amount of material required. 
</p>

<h3 id="Background">
    <B>Objective & Description:</B><a class="anchor-link" href="#Background">&#182;</a>
</h3>

<div style="width: 60%; border-top: 4px solid #00B8C8; border-left: 4px solid #00B8C8; background-color: #FFFFFF; padding: 1em 1em 1em 1em; color: #24292E; margin: 10px 0 20px 0; box-sizing: border-box;">
    <div style="background-color: #00B8C8; color: white; padding: 0.2em 1em; margin: -1em -1em 0em -1em; font-size: 1.2em;"><strong>Project Objective:</strong> To build a machine learning based algorithm for urban classification through street view images</div>
    <p>
    To achieve so this project requires to find the optimal set of nodal coordinates and cross-sectional properties. Achieving so will allow to minimize as much as possible the total weight of the structure, while satisfying a number of constraints relating to the structures natural frequencies. Achieving a low mass solutions that also satisfies the natural frequencies constraints established demonstrates a methodology to make structures more efficient and safe since we are achieving to use less material in a way that still ensures the structural integrity of the structure. Moreover, this project will also aim to explore the efficacy of current optimisation methods and potentially improvements to be achieved from implementing machine learning methods.
    </p>
</div>

## <strong> X | Imports</strong>

In [1]:
import os
import h5py
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from plotly import graph_objects as go

import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split

from streetscapes.models import CNN, VAE, GM

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


## <strong> 0 | Importing the data</strong>

### <strong> 0.1 | Process geojson's</strong>

In [5]:
# --------------------------- IMPORT .geojson FILES -------------------------- #
directory = '_data\\geo_json'
gpd_df = {}

for root, _, files in os.walk(directory):
    for filename in files:
        if filename.endswith('.geojson'):
            gpd_df[filename] = process_geojson(root, filename)


### <strong> 0.2 | Plot geometries on Mapbox</strong>

In [8]:
gj_include = ['boundary.geojson', 'landuse.geojson', 'panoids.geojson']
gpd_add = {key: gpd_df[key] for key in gj_include}

map = KeplerGl(height=600)
for gdf_name in gpd_add:
    map.add_data(gpd_add[gdf_name], name= gdf_name )

file_save = '_maps\\SC_land_preview.html'
map.save_to_html(file_name= file_save, config={
    'mapState': {
        'latitude': 52.01153531997234,
        'longitude': 4.3588424177636185,
        'zoom': 16
    }
})

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to _maps\SC_land_preview.html!


### <strong> 0.3 | Import and process streetview images</strong>

In [2]:
directory = '_data/imagedb'
n_images = 500

transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize the image to your desired dimensions
    transforms.ToTensor(),               # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize the image if necessary
])

labels = []
dataset = []
listdir = os.listdir(directory)
for i in range(n_images):
    filename = listdir[i]
    if filename.endswith('.png'):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r') as image:
            image = Image.open(file_path)
            image_tensor = transform(image)
            dataset.append(image_tensor.to(device))
            labels.append(filename)

## <strong> 1 | Extracting the image features with `ResNet`</strong>

### <strong> 1.1 | Extract the features </strong>

In [3]:
STREETSCAPES01 = CNN()
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
features = STREETSCAPES01.extract_features(dataloader)

c:\Program Files\Python310\@Environments\TUD-JF\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Program Files\Python310\@Environments\TUD-JF\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### <strong> 1.2 | Save features  </strong>

In [2]:
path = '_data\sc_data'
# dataset = features.detach().cpu().numpy()

# Save features and labels
# with h5py.File(os.path.join(path, 'DelftSV_imgfeatuers.h5'), 'w') as hf:
#     hf.create_dataset('features', data= dataset)
#     hf.create_dataset('labels', data=labels)

# Load features and labels
with h5py.File(os.path.join(path, 'DelftSV_imgfeatuers.h5'), 'r') as hf:
    features = hf['features'][:]
    labels = hf['labels'][:]

## <strong> 2 | Dimensionality reduction through a Variational Autoencoder</strong>

### <strong> 2.2 | Initiate the model</strong>

In [3]:
dset = list(zip(features, labels))

# Set rng for reproducability
torch.manual_seed(0)
g = torch.Generator()
g.manual_seed(0)

# Set params for dataset handling
batch_size = 50
test_rat = 0.8
validation_rat = 0.7

# Model parameters
latent_dim = 50

hidden_dim = 512 #Efficient
hidden_num = 5 #Number of layers
epochs = 250
save_model = True

#Early stopping param
patience = 25  # Number of epochs to wait for improvement before stopping
delta = 0.001  # Minimum change to signify an improvement
wait = 0  # Counter for epochs waited since last improvement

In [4]:
# -------------------------------- DATA SPLIT -------------------------------- #
data_test, data_train = random_split(
    dset, [test_rat, 1 - test_rat], generator=g
)
data_train, data_val = random_split(
    list(data_train), [validation_rat, 1 - validation_rat], generator=g
)

# ---------------------------------- LOADERS --------------------------------- #
test_loader = DataLoader(
    data_test, batch_size=batch_size, shuffle=True, generator=g
)
training_loader = DataLoader(
    data_train, batch_size=batch_size, shuffle=True, generator=g
)
validation_loader = DataLoader(
    data_val, batch_size=batch_size, shuffle=True, generator=g
)

# create the model
model = VAE(
    input_dim= features.shape[1],
    latent_dim= latent_dim,
    hidden_dim= hidden_dim,
    hidden_num= hidden_num,
    save_model = save_model,
    path = '_data\\models\\vae'
)

### <strong> 2.3 | Train the model</strong>

In [5]:
# Define optimizer
optimizer = torch.optim.Adam(model.parameters())
loaders = [training_loader, validation_loader]
tloss, vloss = model.train_(optimizer,loaders,epochs,patience,wait)

+---------+--------------+------------+-------------+--------------+
|   Epoch |   Train Loss |   Val Loss |   Best Loss |   Best Epoch |
+=========+==============+============+=============+==============+
|       0 |      731.578 |     600.15 |     731.578 |            0 |
+---------+--------------+------------+-------------+--------------+
+---------+--------------+------------+-------------+--------------+
|   Epoch |   Train Loss |   Val Loss |   Best Loss |   Best Epoch |
+=========+==============+============+=============+==============+
|       1 |      457.461 |    370.848 |     457.461 |            1 |
+---------+--------------+------------+-------------+--------------+
+---------+--------------+------------+-------------+--------------+
|   Epoch |   Train Loss |   Val Loss |   Best Loss |   Best Epoch |
+=========+==============+============+=============+==============+
|       2 |      279.028 |    205.017 |     279.028 |            2 |
+---------+--------------+--------

In [ ]:
# # plot complete training and validation loss history
vloss_cpu = [tensor.cpu() for tensor in vloss]
fig = go.Figure()
fig.add_trace(go.Scatter(y=tloss, mode='lines', name='training loss'))
fig.add_trace(go.Scatter(y=vloss_cpu, mode='lines', name='validation loss'))
fig.update_layout(title='VAE Training and Validation Loss',
                   xaxis_title='Epoch',
                   yaxis_title='Loss',
                   template = "plotly_white")

fig.show()

### <strong> 2.4 | Convert image feature into VAE's latent space</strong>

In [6]:
img_zfeat = []
labels = []

for [data,label] in test_loader:
    x = data.to(model.device)  # Ensure data is on the correct device
    z, mu, var = model.forward(x)
    
    img_zfeat.append(z)
    labels.append(label)
encoded_feat = torch.cat(img_zfeat)

### <strong> 2.5 | Plotting the latent features</strong>

In [8]:
# Plot the features in the a dummy latent space
ubound = encoded_feat.shape[1]
ind_list = np.random.randint(0, ubound, size=(3,))
imgfeature_sp(encoded_feat,ind_list)

NameError: name 'imgfeature_sp' is not defined

## <strong> 3 | Classification through Gaussian Mixtures</strong>

### <strong> 3.2 | Initiate the model</strong>

In [7]:
data = data.cpu()
gmm = GM()
dict_ = gmm.fit(data, k_max= 30, dgrad = 0.03)

In [8]:
from streetscapes.processing.utils.monitor_training import gm_elbo
gm_elbo(**dict_)

NameError: name 'aics' is not defined

In [18]:
z_cpu = features.cpu()
cluster_labels = gmm.predict_proba(z_cpu)
    

In [22]:
cluster_labels[17]

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])

### <strong> 3.3 | Plotting the classification</strong>

#### <strong> Actual classification redued into a 2D space</strong>

#### <strong> Dummy classification a 3D space</strong>

In [149]:
K = 3 
gmm = GaussianMixture(n_components=K, covariance_type='full', random_state=0)
gmm.fit(data)

gmc_space(data, [0,1,2], gmm)

## <strong> 4 | Manual Labelling and benchmarking </strong>

## <strong> O | Other</strong>

In [57]:
def imgfeature_sp(features, latd, lat = False):
        
    z_cpu = features.detach().cpu().numpy()
    fig = plt.figure()
    fig = go.Figure(data=[go.Scatter3d(
        x=z_cpu[:, latd[0]],  # X axis data
        y=z_cpu[:, latd[1]],  # Y axis data
        z=z_cpu[:, latd[2]],  # Z axis data
        mode='markers',
        marker=dict(
            size=5,  
            opacity=0.8,  
        )
    )])

    fig.update_layout(
        title_text = "Image feature distribution in latent space",
        margin=dict(l=0, r=0, b=0, t=0),  
        scene=dict(
            xaxis_title=f'Dimension {latd[0]}', 
            yaxis_title=f'Dimension {latd[1]}', 
            zaxis_title=f'Dimension {latd[2]}'
        ),
        template = 'plotly_dark'
    )

    fig.show()

In [ ]:
def gmc_space(features, latd, gmm):
    """
    Visualizes the Gaussian Mixture Model clusters in 3D with simplified boundaries.
    
    Args:
    - features (torch.Tensor): The latent features from the VAE.
    - latd (list of int): The indices of the dimensions to plot.
    - gmm (GaussianMixture): The fitted Gaussian Mixture Model.
    """
    
    z_cpu = features
    cluster_labels = gmm.predict(z_cpu)
    
    # Define colors for each cluster
    colors = px.colors.qualitative.Plotly

    fig = go.Figure()

    # Plot data points with colors based on cluster membership
    for i in range(gmm.n_components):
        cluster_data = z_cpu[cluster_labels == i]
        fig.add_trace(go.Scatter3d(
            x=cluster_data[:, latd[0]],
            y=cluster_data[:, latd[1]],
            z=cluster_data[:, latd[2]],
            mode='markers',
            marker=dict(size=3, color=colors[i % len(colors)]),
            name=f'Cluster {i+1}'
        ))

    # Add spheres to indicate cluster centers (simplified boundary visualization)
    for mean, color in zip(gmm.means_, colors[:gmm.n_components]):
        sphere = create_sphere(mean[latd[0]], mean[latd[1]], mean[latd[2]], radius=0.5, color=color)
        fig.add_trace(sphere)
    
    # Update plot layout
    fig.update_layout(
        title_text="GMM Clusters in Latent Space",
        scene=dict(
            xaxis_title=f'Dimension {latd[0]}',
            yaxis_title=f'Dimension {latd[1]}',
            zaxis_title=f'Dimension {latd[2]}'
        ),
        template='plotly_dark'
    )
    
    return fig.show()

def create_sphere(x_center, y_center, z_center, radius, color):
    """
    Generates a sphere surface centered at (x_center, y_center, z_center).
    
    Args:
    - x_center, y_center, z_center (float): Center of the sphere.
    - radius (float): Radius of the sphere.
    - color (str): Color of the sphere.
    
    Returns:
    - A Plotly figure object representing the sphere.
    """
    phi = np.linspace(0, 2*np.pi, 20)
    theta = np.linspace(0, np.pi, 20)
    phi, theta = np.meshgrid(phi, theta)
    
    x = radius * np.sin(theta) * np.cos(phi) + x_center
    y = radius * np.sin(theta) * np.sin(phi) + y_center
    z = radius * np.cos(theta) + z_center
    
    return go.Mesh3d(
        x=x.flatten(),
        y=y.flatten(),
        z=z.flatten(),
        alphahull=0,
        opacity=0.2,
        color=color
    )